In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from matplotlib.colors import ListedColormap

#классификация
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.neural_network import MLPClassifier #нейронка
from sklearn.naive_bayes import GaussianNB #наивный байес
from sklearn.neighbors import KNeighborsClassifier # k ближайших соседей

#регрессия
from sklearn.linear_model import LinearRegression #линейная регрессия
from sklearn.svm import SVC #метод опорных векторов
from sklearn.metrics import mean_squared_error


#Оценка качества в задачах классификации
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

#Оценка качества в задачах регрессии
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import preprocessing

In [2]:
#Задача: вычисление значения параметра code на основании остальных параметров.
train_data = pd.read_csv("Набор_3_страны_мира.csv",delimiter=';',decimal=',')#Меняем разделитель на ;
train_data = train_data.dropna()

In [3]:
train_data

,country,region,population,area,coastline,infant_mortality,gdp,literacy,phones,arable,birthrate,deathrate
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,0.00,163.07,700.0,36.0,3.2,12.13,46.60,20.34
1,Albania,EASTERN EUROPE,3581655,28748,1.26,21.52,4500.0,86.5,71.2,21.09,15.11,5.22
2,Algeria,NORTHERN AFRICA,32930091,2381740,0.04,31.00,6000.0,70.0,78.1,3.22,17.14,4.61
3,American Samoa,OCEANIA,57794,199,58.29,9.27,8000.0,97.0,259.5,10.00,22.46,3.27
4,Andorra,WESTERN EUROPE,71201,468,0.00,4.05,19000.0,100.0,497.2,2.22,8.71,6.25
...,...,...,...,...,...,...,...,...,...,...,...,...
218,Venezuela,LATIN AMER. & CARIB,25730435,912050,0.31,22.20,4800.0,93.4,140.1,2.95,18.71,4.92
219,Vietnam,ASIA (EX. NEAR EAST),84402966,329560,1.05,25.95,2500.0,90.3,187.7,19.97,16.86,6.22
224,Yemen,NEAR EAST,21456188,527970,0.36,61.50,800.0,50.2,37.2,2.78,42.89,8.30
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,0.00,88.29,800.0,80.6,8.2,7.08,41.00,19.93


In [4]:
#выбираем рандомное число
my_code = "ВолковНиколай"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [5]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

41


In [6]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, 
                              test_size=val_size, 
                              random_state=random_state)
print(len(train), len(val))

162 41


In [7]:
print(train)

         country                               region  population     area  \
101       Italy   WESTERN EUROPE                         58133509   301230   
197       Syria   NEAR EAST                              18881361   185180   
129       Malta   WESTERN EUROPE                           400214      316   
155        Oman   NEAR EAST                               3102229   212460   
9       Armenia                  C.W. OF IND. STATES      2976372    29800   
..           ...                                  ...         ...      ...   
161        Peru               LATIN AMER. & CARIB        28302603  1285220   
182  Seychelles   SUB-SAHARAN AFRICA                        81541      455   
4       Andorra   WESTERN EUROPE                            71201      468   
188     Somalia   SUB-SAHARAN AFRICA                      8863338   637657   
55     Djibouti   SUB-SAHARAN AFRICA                       486530    23000   

     coastline  infant_mortality      gdp  literacy  phones  ar

In [8]:
print(val)

                   country                               region  population  \
165           Puerto Rico               LATIN AMER. & CARIB         3927188   
77                  Ghana   SUB-SAHARAN AFRICA                     22409572   
37             Cape Verde   SUB-SAHARAN AFRICA                       420979   
7       Antigua & Barbuda               LATIN AMER. & CARIB           69108   
24                Bolivia               LATIN AMER. & CARIB         8989046   
65               Ethiopia   SUB-SAHARAN AFRICA                     74777981   
0             Afghanistan         ASIA (EX. NEAR EAST)             31056997   
13             Azerbaijan                  C.W. OF IND. STATES      7961619   
27                 Brazil               LATIN AMER. & CARIB       188078227   
214         United States   NORTHERN AMERICA                      298444215   
212  United Arab Emirates   NEAR EAST                               2602713   
215               Uruguay               LATIN AMER. 

In [9]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
names=['country','region']
num_columns = ['area','coastline',
               'infant_mortality','gdp','literacy','phones'
               ,'arable','birthrate','deathrate']
ord_columns = ['population']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['area', 'coastline', 'infant_mortality',
                                  'gdp', 'literacy', 'phones', 'arable',
                                  'birthrate', 'deathrate'])])

In [10]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

# даем название столбцам
column_names = num_columns + names + ord_columns
sc_train.columns = column_names
sc_val.columns = column_names

sc_train

,area,coastline,infant_mortality,gdp,literacy,phones,arable,birthrate,deathrate,country,region,population
0,0.017641,0.007598,0.019322,0.479853,0.98301,0.415975,0.494396,0.032919,0.298052,Italy,WESTERN EUROPE,58133509
1,0.010845,0.000302,0.144203,0.051282,0.71966,0.148348,0.448675,0.471225,0.092613,Syria,NEAR EAST,18881361
2,0.000018,0.187783,0.00847,0.315018,0.912621,0.487541,0.500445,0.067449,0.213524,Malta,WESTERN EUROPE,400214
3,0.012442,0.002955,0.091159,0.230769,0.706311,0.082384,0.0,0.666436,0.055862,Oman,NEAR EAST,3102229
4,0.001745,0.0,0.111117,0.054945,0.98301,0.188816,0.312222,0.110037,0.218302,Armenia,C.W. OF IND. STATES,2976372
...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.075268,0.000573,0.156961,0.084249,0.889563,0.076589,0.051414,0.303637,0.1448,Peru,LATIN AMER. & CARIB,28302603
158,0.000027,0.325363,0.07009,0.1337,0.490291,0.253235,0.039495,0.201197,0.147005,Seychelles,SUB-SAHARAN AFRICA,81541
159,0.000027,0.0,0.009317,0.338828,1.0,0.480008,0.039495,0.032689,0.145535,Andorra,WESTERN EUROPE,71201
160,0.037344,0.001417,0.605664,0.0,0.245146,0.01072,0.02971,0.871087,0.527012,Somalia,SUB-SAHARAN AFRICA,8863338


In [11]:
# В качестве входных параметров используем первые 9 числовых параметров,
# в качестве выходного - десятый числовой параметр.
x_train = sc_train[num_columns]
x_val = sc_val[num_columns]

y_train = (sc_train[ord_columns].values).flatten()
y_val = (sc_val[ord_columns].values).flatten()

In [30]:
y_train=np.array(y_train, dtype = np.float64)
y_val=np.array(y_val, dtype = np.float64)

In [31]:
y_val

array([3.92718800e+06, 2.24095720e+07, 4.20979000e+05, 6.91080000e+04,
       8.98904600e+06, 7.47779810e+07, 3.10569970e+07, 7.96161900e+06,
       1.88078227e+08, 2.98444215e+08, 2.60271300e+06, 3.43193200e+06,
       1.61342190e+07, 1.71019000e+05, 1.30139260e+07, 4.07614000e+06,
       1.08004000e+05, 2.81957540e+07, 2.73071340e+07, 6.86884330e+07,
       4.53125000e+05, 2.27972300e+06, 5.21389800e+06, 3.30989320e+07,
       1.09535200e+09, 1.39029720e+07, 1.73407020e+07, 2.83222400e+06,
       5.54870200e+06, 4.46670600e+06, 1.17168290e+07, 6.46315950e+07,
       3.74453920e+07, 1.22935450e+07, 1.47365352e+08, 5.77940000e+04,
       1.32433300e+06, 1.13633400e+06, 9.18398400e+06, 5.23137200e+06,
       1.64156400e+06])

In [32]:
#Создаем модели
models = [LinearRegression(),
              SVC()]

In [33]:
#Обучаем модели
for model in models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
SVC()


In [40]:
# Оценииваем качество работы моделей на валидационной выборке.
MeanSquaredError = []
MeanAbsoluteError = []
R2Score = []
MeanAbsolutePercentageError = []

for model in models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mae = mean_absolute_error(y_val, val_pred)
    r2 = r2_score(y_val, val_pred)
    mape = mean_absolute_percentage_error(y_val, val_pred)
    
    MeanSquaredError.append(mse)
    MeanAbsoluteError.append(mae)
    R2Score.append(r2)
    MeanAbsolutePercentageError.append(mape)
    print(model, '\t', mse,'\t','mean_squared_error')
    print(model, '\t', mae,'\t','mean_absolute_error')
    print(model, '\t', r2,'\t','r2_score')
    print(model, '\t', mape,'\t','mean_absolute_percentage_error')

LinearRegression() 	 2.338312657845762e+16 	 mean_squared_error
LinearRegression() 	 52003490.02996589 	 mean_absolute_error
LinearRegression() 	 0.2278561828220691 	 r2_score
LinearRegression() 	 25.3219517427747 	 mean_absolute_percentage_error
SVC() 	 6.393583669411247e+16 	 mean_squared_error
SVC() 	 85902904.75609756 	 mean_absolute_error
SVC() 	 -1.111251497262911 	 r2_score
SVC() 	 4.185792509720964 	 mean_absolute_percentage_error


In [42]:
MeanSquaredError

[2.338312657845762e+16, 6.393583669411247e+16]

In [47]:
#выбираем лучшую модель
i_max = MeanSquaredError.index(min(MeanSquaredError))
best_c_model = models[i_max]
print(best_c_model,'\t','mean_squared_error')

LinearRegression() 	 mean_squared_error


In [48]:
i_max = MeanAbsoluteError.index(min(MeanAbsoluteError))
best_c_model = models[i_max]
print(best_c_model,'\t','mean_absolute_error')

LinearRegression() 	 mean_absolute_error


In [49]:
i_max = R2Score.index(min(R2Score ))
best_c_model = models[i_max]
print(best_c_model,'\t','r2_score')

SVC() 	 r2_score


In [50]:
i_max = MeanAbsolutePercentageError.index(min(MeanAbsolutePercentageError))
best_c_model = models[i_max]
print(best_c_model,'\t','mean_absolute_percentage_error')

SVC() 	 mean_absolute_percentage_error
